# ADAPT VQE

## Overview
In this notebook, we will implement [ADAPT-VQE](https://www.nature.com/articles/s41467-019-10988-2), to show how to use TenCirChem to build novel algorithms.

## Form Operator Pool

In [1]:
from tencirchem import UCC
from tencirchem.molecule import h4

ucc = UCC(h4)

# get single and double excitations
# param_id maps operators to parameters (some operators share the same parameter)
ex1_ops, ex1_param_ids, _ = ucc.get_ex1_ops()
ex2_ops, ex2_param_ids, _ = ucc.get_ex2_ops()

The meanings of the numbers are best summarized in the following figure. The elements in the tuples are spin-orbital indices, with the first half and the second half corresponds to creation and annihilation operators, respectively.

![excitation](../statics/excitation.png)

In [2]:
list(zip(ex1_ops, ex1_param_ids))

[((6, 4), 0),
 ((2, 0), 0),
 ((7, 4), 1),
 ((3, 0), 1),
 ((6, 5), 2),
 ((2, 1), 2),
 ((7, 5), 3),
 ((3, 1), 3)]

In [3]:
list(zip(ex2_ops, ex2_param_ids))

[((6, 7, 5, 4), 0),
 ((2, 3, 1, 0), 0),
 ((2, 6, 4, 0), 1),
 ((2, 7, 4, 0), 2),
 ((6, 3, 0, 4), 2),
 ((3, 7, 4, 0), 3),
 ((2, 6, 5, 0), 4),
 ((6, 2, 1, 4), 4),
 ((2, 7, 5, 0), 5),
 ((6, 3, 1, 4), 5),
 ((3, 6, 5, 0), 6),
 ((7, 2, 1, 4), 6),
 ((3, 7, 5, 0), 7),
 ((7, 3, 1, 4), 7),
 ((2, 6, 5, 1), 8),
 ((2, 7, 5, 1), 9),
 ((6, 3, 1, 5), 9),
 ((3, 7, 5, 1), 10)]

In [4]:
# group the operators to form an operator pool
# operators with the same parameter are grouped together.
from collections import defaultdict

op_pool = defaultdict(list)
for ex1_op, ex1_id in zip(ex1_ops, ex1_param_ids):
    op_pool[(1, ex1_id)].append(ex1_op)
for ex2_op, ex2_id in zip(ex2_ops, ex2_param_ids):
    op_pool[(2, ex2_id)].append(ex2_op)
op_pool = list(op_pool.values())
op_pool

[[(6, 4), (2, 0)],
 [(7, 4), (3, 0)],
 [(6, 5), (2, 1)],
 [(7, 5), (3, 1)],
 [(6, 7, 5, 4), (2, 3, 1, 0)],
 [(2, 6, 4, 0)],
 [(2, 7, 4, 0), (6, 3, 0, 4)],
 [(3, 7, 4, 0)],
 [(2, 6, 5, 0), (6, 2, 1, 4)],
 [(2, 7, 5, 0), (6, 3, 1, 4)],
 [(3, 6, 5, 0), (7, 2, 1, 4)],
 [(3, 7, 5, 0), (7, 3, 1, 4)],
 [(2, 6, 5, 1)],
 [(2, 7, 5, 1), (6, 3, 1, 5)],
 [(3, 7, 5, 1)]]

## ADAPT-VQE Iteration
Once the operator pool is formed, ADAPT-VQE constructs the ansatz by picking operators in the pool iteratively. The criteria for picking the operators is to maximize the absolute energy gradient if the operator is picked.
Supposing the original ansatz wavefunction is $| \psi \rangle$ and the operator picked is $G_k$, the new ansatz with $G_k$ is

$$
    | \Psi \rangle = e^{\theta_k G_k} | \psi \rangle \ ,
$$

and the energy gradient is
$$
     \frac{\partial \langle E \rangle }{\partial \theta_k} =2 \langle \Psi|HG_k|\Psi \rangle \ .
$$

With $\theta_k = 0$, the gradient is then

$$
    \frac{\partial \langle E \rangle }{\partial \theta_k}\bigg{|}_{\theta_k=0} = 2 \langle \psi|HG_k|\psi \rangle \ .
$$

If multiple operators sharing the same parameter are added at the same time,
then their gradient should be added together.
The iteration stops when the norm of the gradient vector is below a predefined threshold $\epsilon$.
In the following, $| \psi \rangle$ is obtained by `ucc.civector()` as a vector in the configuration interaction space.

In [5]:
import numpy as np

ucc.ex_ops = []
ucc.params = []
ucc.param_ids = []

converged = False
MAX_ITER = 100
EPSILON = 1e-3
for i in range(MAX_ITER):
    print(f"######## Iter {i} ########")
    # calculate gradient of each operator from the pool
    op_gradient_list = []
    psi = ucc.civector()
    bra = ucc.hamiltonian(psi)
    for op_list in op_pool:
        grad = bra.conj() @ ucc.apply_excitation(psi, op_list[0])
        if len(op_list) == 2:
            grad += bra.conj() @ ucc.apply_excitation(psi, op_list[1])
        op_gradient_list.append(2 * grad)
    print("Gradient vector norm: ", np.linalg.norm(op_gradient_list))
    print("Nonzero gradient vector elements:")
    for op_list, g in zip(op_pool, op_gradient_list):
        if np.abs(g) < 1e-10:
            continue
        print(f"{g:.10f}", op_list)
    if np.linalg.norm(op_gradient_list) < EPSILON:
        print(f"Converged. Norm of gradient: {np.linalg.norm(op_gradient_list)}")
        break
    chosen_op_list = op_pool[np.argmax(np.abs(op_gradient_list))]
    print("Picked operator", chosen_op_list)
    if ucc.ex_ops[-len(chosen_op_list) :] == chosen_op_list:
        print(f"Converged. Same operator picked in succession")
        break
    # update ansatz and run calculation
    ucc.ex_ops.extend(chosen_op_list)
    ucc.params = list(ucc.params) + [0]
    ucc.param_ids.extend([len(ucc.params) - 1] * len(chosen_op_list))
    ucc.init_guess = ucc.params
    ucc.kernel()
    print(f"Optimized energy: {ucc.e_ucc}")
else:
    print("Maximum number of iteration reached")

######## Iter 0 ########
Gradient vector norm:  0.6413625239691856
Nonzero gradient vector elements:
0.0000001787 [(6, 4), (2, 0)]
-0.0000001938 [(7, 5), (3, 1)]
0.2103420256 [(6, 7, 5, 4), (2, 3, 1, 0)]
0.2141378225 [(2, 6, 4, 0)]
0.1874444822 [(3, 7, 4, 0)]
0.3756623781 [(2, 7, 5, 0), (6, 3, 1, 4)]
0.1653203524 [(3, 6, 5, 0), (7, 2, 1, 4)]
0.2765500090 [(2, 6, 5, 1)]
0.2029254292 [(3, 7, 5, 1)]
Picked operator [(2, 7, 5, 0), (6, 3, 1, 4)]
Optimized energy: -2.130032459500918
######## Iter 1 ########
Gradient vector norm:  0.5218024526621244
Nonzero gradient vector elements:
-0.0125038973 [(6, 4), (2, 0)]
0.0167842115 [(7, 5), (3, 1)]
0.1713002616 [(6, 7, 5, 4), (2, 3, 1, 0)]
0.2226765539 [(2, 6, 4, 0)]
0.2132785933 [(3, 7, 4, 0)]
0.1071344471 [(3, 6, 5, 0), (7, 2, 1, 4)]
0.3020073191 [(2, 6, 5, 1)]
0.2115115746 [(3, 7, 5, 1)]
Picked operator [(2, 6, 5, 1)]
Optimized energy: -2.1521053433859345
######## Iter 2 ########
Gradient vector norm:  0.4071773446367265
Nonzero gradient vector 

The final ansatz, optimized parameters and reference energies are available by `print_summary`

In [6]:
ucc.print_summary()

################################ Ansatz ###############################
 #qubits  #params  #excitations initial condition
       8        9            14               RHF
############################### Energy ################################
      energy (Hartree)  error (mH) correlation energy (%)
HF           -2.121387   46.173788                 -0.000
MP2          -2.151794   15.766505                 65.854
CCSD         -2.167556    0.004979                 99.989
UCC          -2.167545    0.015250                 99.967
FCI          -2.167561    0.000000                100.000
############################# Excitations #############################
      excitation configuration  parameter  initial guess
0   (2, 7, 5, 0)      10010110  -0.051907      -0.052049
1   (6, 3, 1, 4)      01101001  -0.051907      -0.052049
2   (2, 6, 5, 1)      01010101  -0.140231      -0.139297
3   (3, 7, 4, 0)      10101010  -0.033405      -0.033263
4   (6, 7, 5, 4)      11000011  -0.024488      -0.0